<a href="https://colab.research.google.com/github/anshulk-cmu/CUDA_PageRank/blob/main/CUDA_PageRank_(Google_Search_Engine).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPU env check & log (Colab)
import os, sys, subprocess, datetime, platform
log = []

from datetime import datetime, UTC
ts = datetime.now(UTC).isoformat()

def run(cmd):
    try:
        out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, text=True)
    except subprocess.CalledProcessError as e:
        out = e.output
    return out.strip()

log.append(f"Timestamp: {datetime.now(UTC).isoformat()}Z")
log.append(f"Python: {sys.version.split()[0]}  OS: {platform.platform()}")

# nvidia-smi
log.append("=== nvidia-smi ===")
log.append(run("nvidia-smi || echo 'nvidia-smi not found'"))

# nvcc
log.append("=== nvcc --version ===")
log.append(run("nvcc --version || echo 'nvcc not found'"))

# PyTorch CUDA
try:
    import torch
    log.append(f"PyTorch: {torch.__version__}")
    log.append(f"torch.cuda.is_available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        i = torch.cuda.current_device()
        props = torch.cuda.get_device_properties(i)
        gb = props.total_memory / (1024**3)
        log.append(f"CUDA device count: {torch.cuda.device_count()}")
        log.append(f"Current device index: {i}")
        log.append(f"Device name: {torch.cuda.get_device_name(i)}")
        log.append(f"Compute capability: {props.major}.{props.minor}")
        log.append(f"Total memory (GB): {gb:.2f}")
        log.append(f"CUDA runtime version (torch): {torch.version.cuda}")
    else:
        log.append("CUDA not available in this runtime. Enable GPU in Runtime → Change runtime type.")
except Exception as e:
    log.append(f"PyTorch check failed: {e}")

# Print and save
text = "\n".join(log)
print(text)
with open("/content/gpu_env_log.txt", "w") as f:
    f.write(text)

Timestamp: 2025-09-28T03:50:36.583735+00:00Z
Python: 3.12.11  OS: Linux-6.6.97+-x86_64-with-glibc2.35
=== nvidia-smi ===
Sun Sep 28 03:50:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                  

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /content/drive/MyDrive/cuda-pagerank/data
%cd /content/drive/MyDrive/cuda-pagerank/data

# Download SNAP web-Google (directed web graph)
!wget -nc https://snap.stanford.edu/data/web-Google.txt.gz
!gunzip -kf web-Google.txt.gz

# Quick sanity: show first lines and basic counts
!head -n 5 web-Google.txt
!grep -v '^\s*#' web-Google.txt | wc -l

Mounted at /content/drive
/content/drive/MyDrive/cuda-pagerank/data
--2025-09-28 03:51:11--  https://snap.stanford.edu/data/web-Google.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21168784 (20M) [application/x-gzip]
Saving to: ‘web-Google.txt.gz’

web-Google.txt.gz   100%[===================>]  20.19M  5.24MB/s    in 6.8s    

2025-09-28 03:51:19 (2.97 MB/s) - ‘web-Google.txt.gz’ saved [21168784/21168784]

# Directed graph (each unordered pair of nodes is saved once): web-Google.txt 
# Webgraph from the Google programming contest, 2002
# Nodes: 875713 Edges: 5105039
# FromNodeId	ToNodeId
0	11342
5105039


In [3]:
# Build in-neighbor CSR (rows = dest nodes, cols = source nodes), plus out-degree
import numpy as np, os, gzip, io, time, json

data_path = "/content/drive/MyDrive/cuda-pagerank/data/web-Google.txt"
assert os.path.exists(data_path)

# 1) read edges (skip '#')
src = []
dst = []
with open(data_path, "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        a,b = line.strip().split()
        src.append(int(a)); dst.append(int(b))
src = np.array(src, dtype=np.int64)
dst = np.array(dst, dtype=np.int64)

N = int(max(src.max(), dst.max())) + 1
M = len(src)

# 2) out-degree (for weights 1/outdeg)
outdeg = np.bincount(src, minlength=N).astype(np.int64)

# 3) in-CSR row_ptr/col_idx/val (val = 1/outdeg(source); dangling nodes => 0 later)
row_ptr = np.zeros(N + 1, dtype=np.int64)
np.add.at(row_ptr, dst + 1, 1)
np.cumsum(row_ptr, out=row_ptr)

col_idx = np.empty(M, dtype=np.int64)
val = np.empty(M, dtype=np.float64)

# fill per row using a cursor
cursor = row_ptr[:-1].copy()
w = 1.0 / np.maximum(outdeg, 1)  # avoid div-by-zero
for s, d in zip(src, dst):
    i = cursor[d]
    col_idx[i] = s
    val[i] = w[s]
    cursor[d] = i + 1

# optional: sort columns within each row for better coalescing later
for r in range(N):
    start, end = row_ptr[r], row_ptr[r+1]
    if end - start > 1:
        idx = np.argsort(col_idx[start:end], kind="mergesort")
        col_idx[start:end] = col_idx[start:end][idx]
        val[start:end] = val[start:end][idx]

# persist
outdir = "/content/drive/MyDrive/cuda-pagerank/prep"
os.makedirs(outdir, exist_ok=True)
np.savez_compressed(f"{outdir}/in_csr_webGoogle.npz",
                    N=N, M=M, row_ptr=row_ptr, col_idx=col_idx, val=val, outdeg=outdeg)

# quick log
print(json.dumps({
    "nodes": int(N),
    "edges": int(M),
    "dangling_count": int((outdeg==0).sum()),
    "sample_row0_len": int(row_ptr[1]-row_ptr[0]),
    "saved": f"{outdir}/in_csr_webGoogle.npz"
}, indent=2))

{
  "nodes": 916428,
  "edges": 5105039,
  "dangling_count": 176974,
  "sample_row0_len": 212,
  "saved": "/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz"
}


In [17]:
# CPU PageRank (power method, minimal)
import numpy as np, time, json

print("=== Running CPU PageRank Baseline ===")

# Load pre-processed graph data
npz = np.load("/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz", allow_pickle=False)
N = int(npz["N"]); M = int(npz["M"])
row_ptr = npz["row_ptr"]; col_idx = npz["col_idx"]; val = npz["val"]; outdeg = npz["outdeg"]

# Algorithm parameters
alpha = 0.85
tol = 1e-6
max_iter = 200

# Initialize ranks
r = np.full(N, 1.0 / N, dtype=np.float64)
r_new = np.empty_like(r)

dangling_mask = (outdeg == 0)

# CSR matrix-vector multiplication function
def spmv_pull(row_ptr, col_idx, val, x):
    y = np.zeros_like(x)
    for i in range(N):
        s = 0.0
        start, end = row_ptr[i], row_ptr[i+1]
        for p in range(start, end):
            s += val[p] * x[col_idx[p]]
        y[i] = s
    return y

# Main iteration loop
t0 = time.time()
for it in range(1, max_iter + 1):
    # Perform the SpMV operation
    tmp = spmv_pull(row_ptr, col_idx, val, r)

    # Handle dangling nodes and apply the PageRank formula
    dangling_mass = r[dangling_mask].sum() / N
    r_new[:] = alpha * (tmp + dangling_mass) + (1.0 - alpha) / N

    # Check for convergence
    diff = np.abs(r_new - r).sum()
    r, r_new = r_new, r
    if diff < tol:
        break
t1 = time.time()

# Store results in a JSON string for later analysis
cpu_results = {
    "method": "CPU (NumPy)",
    "iterations": it,
    "total_ms": (t1 - t0) * 1000,
    "ms_per_iter": ((t1 - t0) * 1000) / it,
}

print(f"CPU method completed in {it} iterations.")
print(f"Total time: {cpu_results['total_ms']:.3f} ms")

# Save for final analysis
%env CPU_RESULTS={json.dumps(cpu_results)}

=== Running CPU PageRank Baseline ===
CPU method completed in 62 iterations.
Total time: 210301.528 ms
env: CPU_RESULTS={"method": "CPU (NumPy)", "iterations": 62, "total_ms": 210301.5275001526, "ms_per_iter": 3391.960120970203}


In [18]:
# Save CSR to flat binaries for CUDA C++ (float64 weights)
import numpy as np, json, os

npz = np.load("/content/drive/MyDrive/cuda-pagerank/prep/in_csr_webGoogle.npz", allow_pickle=False)
N = int(npz["N"]); M = int(npz["M"])
row_ptr = npz["row_ptr"].astype(np.int64, copy=False)
col_idx = npz["col_idx"].astype(np.int64, copy=False)
val = npz["val"].astype(np.float64, copy=False)
outdeg = npz["outdeg"].astype(np.int64, copy=False)

outdir = "/content/drive/MyDrive/cuda-pagerank/bin"
os.makedirs(outdir, exist_ok=True)

row_ptr.tofile(f"{outdir}/row_ptr.bin")
col_idx.tofile(f"{outdir}/col_idx.bin")
val.tofile(f"{outdir}/val.bin")
outdeg.tofile(f"{outdir}/outdeg.bin")

with open(f"{outdir}/meta.json", "w") as f:
    json.dump({"N": N, "M": M, "alpha": 0.85, "tol": 1e-6, "max_iter": 200}, f)

print("Saved:", outdir)

Saved: /content/drive/MyDrive/cuda-pagerank/bin


In [19]:
%%writefile /content/pagerank_pull.cu
#include <cstdio>
#include <cstdlib>
#include <cstdint>
#include <vector>
#include <fstream>
#include <string>
#include <iterator>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/transform_reduce.h>
#include <thrust/transform.h>
#include <thrust/functional.h>
#include <thrust/iterator/zip_iterator.h>
#include <thrust/tuple.h>
#include <cmath>
#include <cuda_runtime.h>

// A warp-level reduction helper function.
__device__ inline double warpReduceSum(double val) {
  for (int offset = 16; offset > 0; offset /= 2) {
    val += __shfl_down_sync(0xFFFFFFFF, val, offset);
  }
  return val;
}

// Method 1: Scalar Kernel. Each thread processes one full row.
__global__ void spmv_scalar_kernel(const int64_t* __restrict__ row_ptr,
                                   const int64_t* __restrict__ col_idx,
                                   const double* __restrict__ val,
                                   const double* __restrict__ r,
                                   double* __restrict__ tmp,
                                   int64_t N) {
  int64_t row = blockIdx.x * blockDim.x + threadIdx.x;
  if (row >= N) return;

  double sum = 0.0;
  int64_t start = row_ptr[row];
  int64_t end = row_ptr[row + 1];

  for (int64_t p = start; p < end; ++p) {
    sum += val[p] * r[col_idx[p]];
  }
  tmp[row] = sum;
}

// Method 2: Vector Kernel. Each warp (32 threads) processes one row.
__global__ void spmv_vector_kernel(const int64_t* __restrict__ row_ptr,
                                   const int64_t* __restrict__ col_idx,
                                   const double* __restrict__ val,
                                   const double* __restrict__ r,
                                   double* __restrict__ tmp,
                                   int64_t N) {
  int64_t row = blockIdx.x * blockDim.x + (threadIdx.x / 32);
  if (row >= N) return;

  int lane_id = threadIdx.x % 32;
  double sum = 0.0;
  int64_t start = row_ptr[row];
  int64_t end = row_ptr[row + 1];

  for (int64_t p = start + lane_id; p < end; p += 32) {
    sum += val[p] * r[col_idx[p]];
  }

  sum = warpReduceSum(sum);

  if (lane_id == 0) {
    tmp[row] = sum;
  }
}

// Helper structs for Thrust reductions
struct AbsDiff {
  __host__ __device__ double operator()(const thrust::tuple<double,double>& a) const {
    return fabs(thrust::get<0>(a) - thrust::get<1>(a));
  }
};
struct MaskPick {
  const int64_t* outdeg;
  __host__ __device__ double operator()(const thrust::tuple<double,int64_t>& t) const {
    return (thrust::get<1>(t)==0) ? thrust::get<0>(t) : 0.0;
  }
};

// PageRank finalization kernel
__global__ void finalize_kernel(double* __restrict__ r_new,
                                const double* __restrict__ tmp,
                                double alpha, double d_ave, double base,
                                int64_t N) {
  int64_t i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i >= N) return;
  r_new[i] = alpha * (tmp[i] + d_ave) + base;
}

// Binary file reader for graph data
static void read_bin(const char* path, void* buf, size_t bytes) {
  std::ifstream f(path, std::ios::binary);
  if(!f) { fprintf(stderr, "Cannot open %s\n", path); std::exit(1); }
  f.read(reinterpret_cast<char*>(buf), bytes);
  if(!f) { fprintf(stderr, "Short read on %s\n", path); std::exit(1); }
}

// Generic function to run and benchmark one PageRank method
void run_pagerank(const char* method_name,
                  void (*spmv_kernel)(const int64_t*, const int64_t*, const double*, const double*, double*, int64_t),
                  int64_t N, int max_iter, double tol, double alpha,
                  const thrust::device_vector<int64_t>& d_row_ptr,
                  const thrust::device_vector<int64_t>& d_col_idx,
                  const thrust::device_vector<double>& d_val,
                  const thrust::device_vector<int64_t>& d_outdeg,
                  dim3 blocks, dim3 threads,
                  double& out_ms_total, int& out_iterations, double& out_gbs)
{
    thrust::device_vector<double> r(N, 1.0 / (double)N);
    thrust::device_vector<double> r_new(N, 0.0);
    thrust::device_vector<double> tmp(N, 0.0);

    const double base = (1.0 - alpha) / (double)N;
    double residual = 0.0;
    out_ms_total = 0.0;

    printf("--- Testing %s Method ---\n", method_name);
    printf("Iter | Residual (L1) | Time (ms) | Status\n");
    printf("-----|---------------|-----------|--------\n");

    cudaEvent_t e0, e1;
    cudaEventCreate(&e0); cudaEventCreate(&e1);

    for (out_iterations = 1; out_iterations <= max_iter; ++out_iterations) {
        cudaEventRecord(e0);

        auto zip_dang = thrust::make_zip_iterator(thrust::make_tuple(r.begin(), d_outdeg.begin()));
        auto zip_rpair = thrust::make_zip_iterator(thrust::make_tuple(r.begin(), r_new.begin()));

        spmv_kernel<<<blocks, threads>>>(
            thrust::raw_pointer_cast(d_row_ptr.data()),
            thrust::raw_pointer_cast(d_col_idx.data()),
            thrust::raw_pointer_cast(d_val.data()),
            thrust::raw_pointer_cast(r.data()),
            thrust::raw_pointer_cast(tmp.data()),
            N);

        MaskPick pick{thrust::raw_pointer_cast(d_outdeg.data())};
        double d_mass = thrust::transform_reduce(zip_dang, zip_dang + N, pick, 0.0, thrust::plus<double>());
        double d_ave = d_mass / (double)N;

        finalize_kernel<<< (unsigned)(N + 255)/256, 256 >>>(
            thrust::raw_pointer_cast(r_new.data()),
            thrust::raw_pointer_cast(tmp.data()),
            alpha, d_ave, base, N);

        residual = thrust::transform_reduce(zip_rpair, zip_rpair + N, AbsDiff(), 0.0, thrust::plus<double>());
        r.swap(r_new);

        cudaEventRecord(e1);
        cudaEventSynchronize(e1);
        float ms = 0.0f;
        cudaEventElapsedTime(&ms, e0, e1);
        out_ms_total += ms;

        if (out_iterations == 1 || out_iterations % 10 == 0 || residual < tol) {
            printf("%4d | %12.6e | %8.3f | %s\n",
                   out_iterations, residual, ms,
                   (residual < tol) ? "CONVERGED" : "Running");
        }

        if (residual < tol) break;
    }

    // Bandwidth calculation
    int64_t M = d_col_idx.size();
    const double bytes_per_iter = (double)(M * (sizeof(double) + sizeof(int64_t)) + N * (3 * sizeof(double)));
    double ms_per_iter = out_ms_total / out_iterations;
    out_gbs = (bytes_per_iter / (ms_per_iter / 1000.0)) / 1e9;

    cudaEventDestroy(e0); cudaEventDestroy(e1);
    printf("Convergence: SUCCESS after %d iterations.\n", out_iterations);
    printf("JSON_RESULT: {\"method\":\"GPU %s\",\"iterations\":%d,\"total_ms\":%.3f,\"ms_per_iter\":%.3f,\"gbs\":%.2f}\n\n",
            method_name, out_iterations, out_ms_total, ms_per_iter, out_gbs);
}

int main() {
    printf("=== CUDA PageRank: Scalar vs Vector Kernel Comparison ===\n\n");

    // Config parsing
    std::ifstream mf("/content/drive/MyDrive/cuda-pagerank/bin/meta.json");
    std::string s((std::istreambuf_iterator<char>(mf)), std::istreambuf_iterator<char>());
    auto getnum = [&](const char* key) -> double {
        auto k = s.find(key); if(k == std::string::npos) return 0; k = s.find(':', k); auto e = s.find_first_of(",}\n", k + 1);
        return atof(s.substr(k + 1, e - k - 1).c_str());
    };
    int64_t N = (int64_t)getnum("\"N\"");
    int64_t M = (int64_t)getnum("\"M\"");
    double alpha = getnum("\"alpha\"");
    double tol = getnum("\"tol\"");
    int max_iter = (int)getnum("\"max_iter\"");

    printf("Problem Size:\n");
    printf("  Nodes (web pages): %lld\n", (long long)N);
    printf("  Edges (links):     %lld\n", (long long)M);
    printf("  Avg links/page:    %.2f\n\n", (double)M / (double)N);

    // Load data from disk
    std::vector<int64_t> h_row_ptr(N + 1), h_col_idx(M), h_outdeg(N);
    std::vector<double> h_val(M);
    read_bin("/content/drive/MyDrive/cuda-pagerank/bin/row_ptr.bin", h_row_ptr.data(), (N + 1) * sizeof(int64_t));
    read_bin("/content/drive/MyDrive/cuda-pagerank/bin/col_idx.bin", h_col_idx.data(), M * sizeof(int64_t));
    read_bin("/content/drive/MyDrive/cuda-pagerank/bin/val.bin", h_val.data(), M * sizeof(double));
    read_bin("/content/drive/MyDrive/cuda-pagerank/bin/outdeg.bin", h_outdeg.data(), N * sizeof(int64_t));

    // Transfer data to GPU
    thrust::device_vector<int64_t> d_row_ptr = h_row_ptr;
    thrust::device_vector<int64_t> d_col_idx = h_col_idx;
    thrust::device_vector<int64_t> d_outdeg = h_outdeg;
    thrust::device_vector<double> d_val = h_val;

    double ms_scalar = 0, ms_vector = 0, gbs_scalar = 0, gbs_vector = 0;
    int iter_scalar = 0, iter_vector = 0;

    // --- Run Scalar Method ---
    int threads_scalar = 256;
    int blocks_scalar = (N + threads_scalar - 1) / threads_scalar;
    run_pagerank("Scalar", spmv_scalar_kernel, N, max_iter, tol, alpha,
                 d_row_ptr, d_col_idx, d_val, d_outdeg,
                 dim3(blocks_scalar), dim3(threads_scalar),
                 ms_scalar, iter_scalar, gbs_scalar);

    // --- Run Vector Method ---
    int threads_vector = 256;
    int warps_per_block = threads_vector / 32;
    int blocks_vector = (N + warps_per_block - 1) / warps_per_block;
    run_pagerank("Vector", spmv_vector_kernel, N, max_iter, tol, alpha,
                 d_row_ptr, d_col_idx, d_val, d_outdeg,
                 dim3(blocks_vector), dim3(threads_vector),
                 ms_vector, iter_vector, gbs_vector);

    return 0;
}

Writing /content/pagerank_pull.cu


In [20]:
# Compile and run CUDA code
!nvcc -O3 -arch=sm_75 /content/pagerank_pull.cu -o /content/pagerank_pull
# The 'tee' command saves the output to a file and also prints it
!/content/pagerank_pull | tee /content/gpu_results.log

=== CUDA PageRank: Scalar vs Vector Kernel Comparison ===

Problem Size:
  Nodes (web pages): 916428
  Edges (links):     5105039
  Avg links/page:    5.57

--- Testing Scalar Method ---
Iter | Residual (L1) | Time (ms) | Status
-----|---------------|-----------|--------
   1 | 8.508467e-01 |    3.446 | Running
  10 | 1.090636e-02 |    3.298 | Running
  20 | 1.298404e-03 |    3.322 | Running
  30 | 2.066399e-04 |    3.264 | Running
  40 | 3.607447e-05 |    3.207 | Running
  50 | 6.557848e-06 |    2.947 | Running
  60 | 1.219864e-06 |    2.993 | Running
  62 | 8.729468e-07 |    2.972 | CONVERGED
Convergence: SUCCESS after 62 iterations.
JSON_RESULT: {"method":"GPU Scalar","iterations":62,"total_ms":197.060,"ms_per_iter":3.178,"gbs":32.62}

--- Testing Vector Method ---
Iter | Residual (L1) | Time (ms) | Status
-----|---------------|-----------|--------
   1 | 6.913818e-01 |    0.737 | Running
  10 | 4.030835e-07 |    0.682 | CONVERGED
Convergence: SUCCESS after 10 iterations.
JSON_RESUL

In [21]:
# Final Analysis
import json
import os
import pandas as pd

# List to hold all results
results = []

# Get CPU results from environment variable
cpu_results_str = os.environ.get('CPU_RESULTS')
if cpu_results_str:
    cpu_data = json.loads(cpu_results_str)
    cpu_data['gbs'] = 'N/A'  # Bandwidth not measured for CPU
    results.append(cpu_data)

# Parse GPU results from the log file
with open('/content/gpu_results.log', 'r') as f:
    for line in f:
        if line.startswith('JSON_RESULT:'):
            # Extract the JSON part of the string
            json_str = line.replace('JSON_RESULT: ', '').strip()
            results.append(json.loads(json_str))

# Create a pandas DataFrame for nice formatting
df = pd.DataFrame(results)

# Calculate speedup relative to the CPU
if not df[df['method'] == 'CPU (NumPy)'].empty:
    cpu_time_per_iter = df[df['method'] == 'CPU (NumPy)']['ms_per_iter'].iloc[0]
    df['speedup'] = (cpu_time_per_iter / df['ms_per_iter']).round(2)
else:
    df['speedup'] = 'N/A'

# Format and display the final table
df_display = df[['method', 'iterations', 'total_ms', 'ms_per_iter', 'gbs', 'speedup']]
df_display = df_display.rename(columns={
    'method': 'Implementation',
    'iterations': 'Iterations',
    'total_ms': 'Total Time (ms)',
    'ms_per_iter': 'Time / Iter (ms)',
    'gbs': 'Bandwidth (GB/s)',
    'speedup': 'Speedup vs CPU'
})

# Display the results
print("=== Overall Performance Analysis ===")
display(df_display.style.format({
    'Total Time (ms)': '{:.2f}',
    'Time / Iter (ms)': '{:.3f}',
}).hide(axis="index"))

=== Overall Performance Analysis ===


Implementation,Iterations,Total Time (ms),Time / Iter (ms),Bandwidth (GB/s),Speedup vs CPU
CPU (NumPy),62,210301.53,3391.960,N/A,1.000000
GPU Scalar,62,197.06,3.178,32.620000,1067.330000
GPU Vector,10,6.96,0.696,148.920000,4873.510000
